In [1]:
import autograd
from autograd import numpy as np

In [2]:
from collections import OrderedDict
import paragami

In [3]:
import time

a = np.full(100000, float('nan'))
b = np.random.random(a.shape)

def set_inplace(b, a):
    a[:] = b + 2
    return a

def set_notinplace(b):
    return b + 2

set_inplace(b, a)

t1 = time.time()
for iter in range(1000):
    set_inplace(b, a)
print('Inplace: ', time.time() - t1)

t1 = time.time()
for iter in range(1000):
    a = set_notinplace(b)
print('Not in place: ', time.time() - t1)

# WTF?  Try %memit and id

Inplace:  0.10280919075012207
Not in place:  0.03980541229248047


In [4]:


def check_dict_equal(dict1, dict2):
    assert dict1.keys() == dict2.keys()
    for key  in dict1:
        assert_array_almost_equal(dict1[key], dict2[key])

dict_pattern = paragami.OrderedDictPattern()
dict_pattern['a'] = paragami.ArrayPattern((2, 3, 4), lb=-1, ub=2)
dict_pattern['b'] = paragami.ArrayPattern((5, ), lb=-1, ub=10)
dict_pattern['c'] = paragami.ArrayPattern((5, 2), lb=-1, ub=10)

dict_val = dict_pattern.random()
print(dict_val)
dict_val2 = dict(dict_val)
print(dict_val2)
flat_val1 = dict_pattern.flatten(dict_val, free=True)
flat_val2 = dict_pattern.flatten(dict_val2, free=True)

np.linalg.norm(flat_val1 - flat_val2)

OrderedDict([('a', array([[[0.89358177, 0.89658284, 1.1121885 , 0.70436588],
        [0.83263673, 1.13825349, 0.90575051, 0.85797621],
        [0.89577067, 1.0274987 , 0.61030241, 0.77511769]],

       [[0.62469266, 1.17655873, 0.71461202, 1.08348495],
        [0.87832466, 1.10244331, 1.0606273 , 0.68987249],
        [1.15039823, 0.53624353, 0.90259403, 0.88781202]]])), ('b', array([6.67444852, 6.02320974, 4.63945515, 4.95775229, 4.87335276])), ('c', array([[6.33539152, 6.49540207],
       [5.32714664, 5.67921865],
       [5.74776878, 4.56019538],
       [4.51882782, 6.80849584],
       [6.55545194, 5.78203494]]))])
{'c': array([[6.33539152, 6.49540207],
       [5.32714664, 5.67921865],
       [5.74776878, 4.56019538],
       [4.51882782, 6.80849584],
       [6.55545194, 5.78203494]]), 'a': array([[[0.89358177, 0.89658284, 1.1121885 , 0.70436588],
        [0.83263673, 1.13825349, 0.90575051, 0.85797621],
        [0.89577067, 1.0274987 , 0.61030241, 0.77511769]],

       [[0.62469266, 1

0.0

In [5]:
param_dict = dict_pattern.random()

def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

test_function(param_dict)

35.14800648802951

In [6]:
class PatternedFunction:
    def __init__(self, original_fun, pattern, free, argnum=0):
        self._fun = original_fun
        self._argnum = argnum
        self.free = free
        self._pattern = pattern
        
    def __str__(self):
        return('Function: {}\nargnum: {}\nfree: {}\npattern: {}'.format(
            self._fun, self._argnum, self.free, self._pattern))
    
    def __call__(self, *args, **kwargs):
        flat_val = args[self._argnum]
        folded_val = self._pattern.fold(flat_val, free=self.free)
        new_args = args[0:self._argnum] + (folded_val, ) + args[self._argnum + 1:-1]
        return self._fun(*new_args, **kwargs)

    
patterned_test_function = PatternedFunction(test_function, dict_pattern, True)
print(patterned_test_function)

flat_param_dict = dict_pattern.flatten(param_dict, free=True)
patterned_test_function(flat_param_dict) - test_function(param_dict)

Function: <function test_function at 0x7fe8a1a05f28>
argnum: 0
free: True
pattern: OrderedDict:
	[a] = Array (2, 3, 4) (lb=-1, ub=2)
	[b] = Array (5,) (lb=-1, ub=10)
	[c] = Array (5, 2) (lb=-1, ub=10)


0.0

In [8]:
patterned_test_function_grad = autograd.grad(patterned_test_function)
patterned_test_function_grad(flat_param_dict)

[[0.81361886 0.22538117 0.98482184]
 [0.58187934 0.33741591 0.29781347]
 [0.97099098 0.23581807 0.80127502]]
0.429360845062225


array([0.03212793, 0.04840759, 0.05202783, 0.05837063, 0.05719376,
       0.04569522, 0.05536747, 0.05306069, 0.05400606, 0.04750543,
       0.03409702, 0.05701079, 0.05344063, 0.05066153, 0.03737994,
       0.04303505, 0.04201406, 0.04715034, 0.05075709, 0.04502291,
       0.0532399 , 0.05701906, 0.05196926, 0.03859918, 5.60911406,
       5.95767544, 5.98015571, 6.1706432 , 6.04188541, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])